In [ ]:
# standard libraries
import importlib
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy import linalg, stats, optimize
import time
import networkx as nx
import sys
sys.path.append('/Users/francescacrucinio/Documents/MD_LVM')

import md_lvm
importlib.reload(md_lvm)

In [ ]:
nodes = 100
pi = np.array([0.6, 0.4])
probs = np.array([[0.25, 0.1], [0.1, 0.2]])

In [ ]:
def sbm_sample(pi, nu, nodes):
    # block allocation
    x = np.random.choice(np.arange(len(pi)), nodes, p=pi)
    # adjacency matrix 
    A = np.zeros((nodes, nodes))
    for i in range(nodes):
        A[i, np.delete(np.arange(nodes), i)] = np.random.binomial(1, p = probs[x[i], np.delete(x, i)], size = nodes-1)
    return A, x

In [ ]:
y, res_z = sbm_sample(pi, probs, nodes)
# y = np.loadtxt("adj.txt")

In [ ]:
th0 = np.array([0.5, 0.5, 0.5, 0.5])  # Initial parameter guess.
X0 = np.random.binomial(1, 0.5, nodes)+1  # Initial particle cloud.

In [ ]:
def sbm_saem_proposal(data, v, theta_current):
    N = v.size
    prop = 2*np.random.binomial(1, 0.5, N)-1
    log_acceptance = ll_sbm(theta_current, prop, data) - ll_sbm(theta_current, v, data)
    accepted = np.log(np.random.uniform(size = v.shape[0])) <= log_acceptance
    output = v.copy()
    output[accepted] = prop[accepted]
    return output
def sbm_saem_sufficient_stat(v, data):
    s1 = np.sum(v == 1)
    s2 = np.zeros(3)
    s3 = np.zeros(3)
    for i in range(N):
        for j in range(N):
            if(j != i):
                if(x[i] == 1):
                    if(x[j] == 1):
                        s2[0] += y[i,j] + y[j,i]
                        s3[0] += (1-y[i,j]) + (1-y[j,i])
                    if(x[j] == 2):
                        s2[1] += y[i,j] + y[j,i]
                        s3[1] += (1-y[i,j]) + (1-y[j,i])
                if(x[i] == 2):
                    if(x[j] == 1):
                        s2[1] += y[i,j] + y[j,i]
                        s3[1] += (1-y[i,j]) + (1-y[j,i])
                    if(x[j] == 2):
                        s2[2] += y[i,j] + y[j,i]
                        s3[2] += (1-y[i,j]) + (1-y[j,i])
    return s1, s2, s3
def sbm_saem_mle(s1, s2, s3, N):
    pq = s1/N
    nu = s2/(s2+s3)
    return pq, nu